In [ ]:
import psycopg2
import json
from flask import Flask, request, jsonify
from flask_cors import CORS

In [ ]:
class DictionaryDB:
    instance_count = 0
    connection = None

    def __init__(self):
        DictionaryDB.instance_count += 1
        self.cursor = None
        self.connection = DictionaryDB.getConnection()

    @classmethod
    def getConnection(cls):
        if cls.connection is None:
            with open("credentials.json", "r") as file:
                credentials = json.load(file)

            cls.connection = psycopg2.connect(
                database=credentials["database"],
                user=credentials["user"],
                password=credentials["password"],
                host=credentials["host"],
                port=credentials["port"],
            )
        return cls.connection

    def getCursor(self):
        if self.cursor is None:
            self.cursor = self.connection.cursor()
        return self.cursor

    def getWordStr(self, word1):
        if word1 is None:
            return WordData(None, None, None)

        cursor = self.getCursor()
        cursor.execute(
            "SELECT id, word, language FROM Words WHERE word = %s",
            (word1,),  # Agrega una coma para crear una tupla con un solo elemento
        )

        word_data = []
        for id, word, language in cursor.fetchall():
            word_data.append(WordData(word, language, id))
        return word_data
        
    def getWord(self, word, language):
        if word is None or language is None:
            return WordData(None, None, None)

        cursor = self.getCursor()
        cursor.execute(
            "SELECT id, word, language FROM Words WHERE word = %s AND language = %s",
            (word, language),
        )
        result = cursor.fetchone()
        if result:
            return WordData(result[1], result[2], result[0])
        else:
            return WordData(None, None, None)
        
    def getWordByID(self, wordId):
        if wordId is None:
            return WordData(None, None, None)

        cursor = self.getCursor()
        cursor.execute("SELECT id, word, language FROM Words WHERE id = %s", (wordId,))

        result = cursor.fetchone()
        if result:
            return WordData(result[1], result[2], result[0])
        else:
            return WordData(None, None, None)

    def getAllWords(self):
        cursor = self.getCursor()
        cursor.execute("SELECT id, word, language FROM Words")

        all_word_data = []
        for id, word, language in cursor.fetchall():
            all_word_data.append(WordData(word, language, id))

        return all_word_data

    def getLanguageDictionary(self, language):
        if language is None:
            return []

        cursor = self.getCursor()
        cursor.execute("SELECT id, word FROM Words WHERE language = %s", (language,))
        return [WordData(word, language, id) for id, word in cursor.fetchall()]

    def getWordRelationships(self, word_data_instance):
        if word_data_instance is None or word_data_instance.id is None:
            return []

        cursor = self.getCursor()
        cursor.execute(
            "SELECT * FROM getWordRelationships(%s)", (word_data_instance.id,)
        )

        word_relationship_data = []
        for relationship_id, word_id, word, language in cursor.fetchall():
            word_relationship_data.append(
                (relationship_id, WordData(word, language, word_id))
            )

        return word_relationship_data

    def getAllErrors(self):
        cursor = self.getCursor()
        cursor.execute("SELECT * FROM getAllErrors()")

        return [
            ErrorData(
                word=row[1],
                times=row[3],
                id=row[0],
                wordData=WordData(word=row[6], language=row[4], id=row[5]),
            )
            for row in cursor.fetchall()
        ]

    def getErrorsByStr(self, word):
        cursor = self.getCursor()
        cursor.execute("SELECT * FROM getErrorsByStr(%s)", (word,))

        return [
            ErrorData(
                word=row[1],
                times=row[3],
                id=row[0],
                wordData=WordData(word=row[6], language=row[4], id=row[5]),
            )
            for row in cursor.fetchall()
        ]

    def addLanguage(self, language):
        if language is None:
            return

        cursor = self.getCursor()
        cursor.execute("SELECT addLanguage(%s)", (language,))

        self.connection.commit()

    def addWord(self, wordData):
        if wordData is None or wordData.word is None or wordData.language is None:
            return None  # Retorna None si los datos son inválidos

        cursor = self.getCursor()
        cursor.execute("SELECT addWord(%s, %s) RETURNING id", (wordData.word, wordData.language))

        self.connection.commit()

        # Recupera el ID de la palabra insertada
        word_id = cursor.fetchone()[0]

        return word_id

    def addWordRelationship(self, wordData1, wordData2):
        if (
            wordData1 is None
            or wordData2 is None
            or wordData1.id is None
            or wordData2.id is None
        ):
            return

        cursor = self.getCursor()
        cursor.execute(
            "SELECT addWordRelationship(%s, %s)", (wordData1.id, wordData2.id)
        )
        self.connection.commit()

    def addWordAndRelationship(self, related_word_data, new_word_data):
        cursor = self.getCursor()
        cursor.execute(
            "CALL addWordAndRelationship(%s, %s, %s)",
            (related_word_data.id, new_word_data.word, new_word_data.language)
        )
        self.connection.commit()
        
    def addTranslation(self, wordData1, wordData2):
        if (
            wordData1 is None
            or wordData2 is None
            or wordData1.word is None
            or wordData1.language is None
            or wordData2.word is None
            or wordData2.language is None
        ):
            return  # Retorna None si los datos son inválidos

        cursor = self.getCursor()
        cursor.execute(
            "SELECT addTranslation(%s, %s, %s, %s)",
            (wordData1.word, wordData1.language, wordData2.word, wordData2.language),
        )
        self.connection.commit()
        """
        with self.conn.cursor() as cur:
            cur.execute(
                "SELECT addTranslation(%s, %s, %s, %s)",
                (wordData1.word, wordData1.language, wordData2.word, wordData2.language)
            )
        self.conn.commit()
        """


    def addError(self, errorData):
        if (
            errorData is None
            or errorData.word is None
            or errorData.wordData is None
            or errorData.wordData.id is None
        ):
            return

        cursor = self.getCursor()
        cursor.execute(
            "SELECT addError(%s, %s)", (errorData.word, errorData.wordData.id)
        )
        self.connection.commit()

    def closeCursor(self):
        if self.cursor is not None:
            self.cursor.close()
            self.cursor = None

    def __del__(self):
        DictionaryDB.instance_count -= 1
        # Aquí retiré los comentarios que estaban desactivando el cierre de la conexión
        """self.closeCursor()
        if DictionaryDB.instance_count == 0 and DictionaryDB.connection is not None:
            DictionaryDB.connection.close()
            DictionaryDB.connection = None
            print("La última instancia está siendo destruida, conexión cerrada.")"""

In [ ]:
class WordData:
    def __init__(self, word, language=None, id=None):
        self.language = language
        self.id = id
        self.word = word

    def __repr__(self):
        return f"WordData(language={self.language}, id={self.id}, word={self.word})"

    def nearestWord(self, dictionary):
        results = [
            (word_data, self.levenshtein(self.word, word_data.word))
            for word_data in dictionary
        ]
        return sorted(results, key=lambda x: x[1])

    def nearestWordTop(self, dictionary):
        results = self.nearestWord(dictionary)
        if not results:
            return []

        min_distance = results[0][1]
        return [word_data for word_data in results if word_data[1] == min_distance]

    @classmethod
    def nearestWordStr(cls, word, dictionary):
        results = [
            (word_data, cls.levenshtein(word, word_data.word))
            for word_data in dictionary
        ]
        return sorted(results, key=lambda x: x[1])

    @classmethod
    def nearestWordTopStr(cls, word, dictionary):
        results = cls.nearestWordStr(word, dictionary)
        if not results:
            return []

        min_distance = results[0][1]
        return [word_data for word_data in results if word_data[1] == min_distance]

    @staticmethod
    def levenshtein(str1, str2):
        m, n = len(str1), len(str2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(m + 1):
            for j in range(n + 1):
                if i == 0:
                    dp[i][j] = j
                elif j == 0:
                    dp[i][j] = i
                elif str1[i - 1] == str2[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1]
                else:
                    dp[i][j] = 1 + min(dp[i - 1][j], dp[i][j - 1], dp[i - 1][j - 1])

        return dp[m][n]
    
    def serialize(self):
        return {
            "wordId": self.id,
            "word": self.word,
            "language": self.language,
        }
    
    def isNone(self):
        if self.word == None and self.language == None and self.id == None:
            return True
        return False





In [ ]:
class ErrorData:
    def __init__(self, word=None, times=None, id=None, wordData=None):
        self.word = word
        self.times = times
        self.id = id
        self.wordData = wordData

    def __repr__(self):
        return f"ErrorData(word={self.word}, times={self.times}, id={self.id}, word_data_instance={self.wordData})"

In [ ]:
dictionary_db = DictionaryDB()

In [ ]:
app = Flask(__name__)
CORS(app)

# Endpoint para buscar una palabra
@app.route('/lookForWord', methods=['POST'])
def look_for_word():
    request_data = request.get_json()
    word = request_data.get('word')
    language = request_data.get('language')

    response_words = []
    not_founded = True
    possible_common_errors = []  # Almacenar sugerencias de errores comunes
    possible_levenshtein_errors = []  # Almacenar sugerencias de errores Levenshtein

    if language == None or language == '':
        # Realiza la búsqueda de la palabra en la base de datos
        possible_words = dictionary_db.getWordStr(word)
        if len(possible_words) > 0:
            for word_data in possible_words:
                translations = dictionary_db.getWordRelationships(word_data)
                response_words.append({
                    "mainword": word_data.serialize(),
                    "translations": [translation[1].serialize() for translation in translations]
                })
            response_data = {
                "words": response_words
            }
            not_founded = False
    else:
        # Realiza la búsqueda de la palabra y LENGUAJE en la base de datos
        word_data = dictionary_db.getWord(word, language)
        if word_data.isNone() == False:
            translations = dictionary_db.getWordRelationships(word_data)
            response_words.append({
                "mainword": word_data.serialize(),
                "translations": [translation[1].serialize() for translation in translations]
            })
            response_data = {
                "words": response_words
            }
            not_founded = False
        
    # Busca errores previamente capturados si no se encontraron
    possible_common_errors = dictionary_db.getErrorsByStr(word)
    common_error_suggestions = [error.wordData.serialize() for error in possible_common_errors]

    # Busca errores analizando el diccionario si no se encontraron
    possible_levenshtein_errors = WordData.nearestWordTopStr(word, dictionary_db.getAllWords())
    levenshtein_error_suggestions = [word[0].serialize() for word in possible_levenshtein_errors]

    if not_founded == True:
        # Combinar sugerencias de errores comunes y Levenshtein si ambos tipos están presentes
        if possible_common_errors and possible_levenshtein_errors:
            combined_suggestions = common_error_suggestions + levenshtein_error_suggestions

            # Mantén el orden original y elimina duplicados
            unique_suggestions_set = set()
            unique_suggestions = []

            for suggestion in combined_suggestions:
                suggestion_tuple = tuple(suggestion.items())
                if suggestion_tuple not in unique_suggestions_set:
                    unique_suggestions_set.add(suggestion_tuple)
                    unique_suggestions.append(dict(suggestion_tuple))

            # Asegúrate de que no haya más de 10 elementos
            unique_suggestions = unique_suggestions[:10]

            response_data = {
                "suggestions": unique_suggestions,
                "type": "both",
                "most_repeated": possible_common_errors[0].times,
                "best_similarity": possible_levenshtein_errors[0][1]
            }
            not_founded = False
        elif possible_common_errors:
            # Si hay sugerencias de errores comunes
            response_data = {
                "suggestions": common_error_suggestions[:10],
                "type": "common_error",
                "most_repeated": possible_common_errors[0].times
            }
            not_founded = False
        elif possible_levenshtein_errors:
            # Si hay sugerencias de errores Levenshtein
            # Filtra si la primera palabra es lo suficientemente cercana
            if possible_levenshtein_errors[0][1] <= 3:
                response_data = {
                    "suggestions": levenshtein_error_suggestions[:10],
                    "type": "levenshtein_error",
                    "best_similarity": possible_levenshtein_errors[0][1]
                }
                not_founded = False

    if not_founded == True:
        # Respuesta no encontrada si no hay sugerencias válidas
        response_data = {
            "suggestions": [],
            "type": "not_founded"
        }
        
    print(response_data)    
    # Construye la respuesta en formato JSON
    return jsonify(response_data)


# Endpoint para agregar una palabra
@app.route('/addWord', methods=['POST'])
def add_word():
    request_data = request.get_json()
    word = request_data.get('word')
    language = request_data.get('language')
    
    # Agrega la palabra a la base de datos
    dictionary_db.addWord(WordData(word, language))
    
    # Construye la respuesta en formato JSON
    response_data = {"request": "addWord", "status": True}
    
    return jsonify(response_data)

# Endpoint para agregar una traducción
@app.route('/addTranslation', methods=['POST'])
def add_translation():
    request_data = request.get_json()
    word_a = request_data.get('word_a')
    language_a = request_data.get('language_a')
    word_b = request_data.get('word_b')
    language_b = request_data.get('language_b')
    
    # Utiliza la nueva función addTranslation para agregar ambas palabras y su relación
    dictionary_db.addTranslation(WordData(word_a, language_a, None), WordData(word_b, language_b, None))
    
    # Construye la respuesta en formato JSON
    response_data = {"request": "addTranslation", "status": True}
    
    return jsonify(response_data)

# Endpoint para agregar un error
@app.route('/addError', methods=['POST'])
def addError():
    request_data = request.get_json()
    error = request_data.get('error')
    wordId = request_data.get('wordId')
    
    # Agrega la palabra a la base de datos
    dictionary_db.addError(ErrorData(error,None,None,WordData(None,None,wordId)))
    # Construye la respuesta en formato JSON
    response_data = {"request": "addError", "status": True}
    
    return jsonify(response_data)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [ ]:
#Prepare words
words = [
    "casa", "perro", "gato", "coche", "manzana", "amarillo", "rojo", "azul", "verde", "jugar",
    "correr", "saltar", "comer", "beber", "sol", "luna", "estrella", "montaña", "playa", "río",
    "ciudad", "campo", "amigo", "familia", "trabajo", "estudiar", "libro", "papel", "lapiz", "telefono",
    "computadora", "internet", "musica", "pelicula", "cancion", "arte", "dibujar", "pintar", "viajar",
    "avion", "tren", "bicicleta", "helado", "pizza", "hamburguesa", "ensalada", "ropa", "zapatos",
    "sombrero", "reloj", "maleta", "dinero", "tarjeta", "llave", "puerta", "ventana", "mesa", "silla",
    "cama", "television", "radio", "noticia", "tiempo", "clima", "lluvia", "nieve", "viento", "calor",
    "frio", "primavera", "verano", "otoño", "invierno", "tristeza", "alegria", "enojo", "amor", "odio",
    "paz", "guerra", "politica", "religion", "historia", "geografia", "matematicas", "ciencia", "tecnologia",
    "ingenieria", "sistema", "programacion", "algoritmo", "red", "seguridad", "aplicacion",
    "desarrollo", "proyecto", "equipo", "cliente", "producto", "servicio", "empresa", "negocio", "trabajador",
    "jefe", "oficina", "edificio", "fabrica", "industria", "comercio", "cliente", "venta", "compra", "precio",
    "descuento", "publicidad", "marketing", "comunicacion", "informacion", "documento", "archivo", "imagen",
    "video", "audio", "texto", "pagina", "sitio", "social", "correo", "mensaje", "telefono", "llamada",
    "chat", "videoconferencia", "reunion", "agenda", "cita", "evento", "feria", "conferencia", "exposicion",
    "concierto", "teatro", "pintura", "escultura", "fotografia", "literatura", "poesia", "novela", "cuento",
    "historieta", "revista", "periodico", "noticia", "reportaje", "entrevista", "opinion", "editorial", "critica",
    "deporte", "futbol", "baloncesto", "tenis", "natacion", "atletismo", "gimnasia", "boxeo", "arte",
    "musica", "rock", "pop", "electronica", "jazz", "blues", "reggae", "folklore", "danza",
    "ballet", "flamenco", "salsa", "merengue", "tango", "cumbia", "bachata", "samba", "rumba", "juego", "videojuego",
    "ajedrez", "cartas", "domino", "puzzle", "casino", "apuesta", "ganar", "perder", "victoria", "derrota",
    "emocion", "adrenalina", "estrategia", "habilidad", "diversion", "ocio", "vacacion", "turismo", "aventura",
    "naturaleza", "animal", "planta", "arbol", "flor", "fruta", "vegetal", "carnivoro", "herbivoro", "omnivoro",
    "selva", "desierto", "monte", "campo", "mar", "oceano", "rio", "lago", "isla", "playa", "costa", "ciudad",
    "pueblo", "aldea", "capital", "calle", "avenida", "plaza", "parque", "jardin", "zoo", "museo", "teatro",
    "cine", "biblioteca", "escuela", "universidad", "profesor", "alumno", "examen", "tarea", "clase", "leccion",
    "conferencia", "seminario", "diploma", "certificado", "conocimiento", "aprendizaje", "investigacion",
    "descubrimiento", "invento", "tecnica", "tecnologia", "ingenieria", "avance", "innovacion", "progreso",
    "desarrollo", "proyecto", "equipo", "cliente", "producto", "servicio", "empresa", "negocio", "trabajador",
    "jefe", "oficina", "edificio", "fabrica", "industria", "comercio", "cliente", "venta", "compra", "precio",
    "descuento", "publicidad", "marketing", "comunicacion", "informacion", "documento", "archivo", "imagen",
    "video", "audio", "texto", "pagina", "web", "social", "correo", "mensaje", "telefono", "llamada",
    "chat", "videoconferencia", "reunion", "agenda", "cita", "evento", "feria", "conferencia", "exposicion",
    "concierto", "teatro", "pintura", "escultura", "fotografia", "literatura", "poesia", "novela", "cuento",
    "historieta", "revista", "periodico", "noticia", "reportaje", "entrevista", "opinion", "editorial", "critica",
    "deporte", "futbol", "baloncesto", "tenis", "natacion", "atletismo", "gimnasia", "boxeo", "marcial",
    "clasica", "rock", "pop", "electronica", "jazz", "blues", "reggae", "folklore", "danza",
    "ballet", "flamenco", "salsa", "merengue", "tango", "cumbia", "bachata", "samba", "rumba", "juego", "videojuego",
    "ajedrez", "cartas", "domino", "puzzle", "casino", "apuesta", "ganar", "perder", "victoria", "derrota",
    "emocion", "adrenalina", "estrategia", "habilidad", "diversion", "ocio", "vacacion", "turismo", "aventura",
    "naturaleza", "animal", "planta", "arbol", "flor", "fruta", "vegetal", "carnivoro", "herbivoro", "omnivoro",
    "selva", "desierto", "monte", "campo", "mar", "oceano", "rio", "lago", "isla", "playa", "costa", "ciudad",
    "pueblo", "aldea", "capital", "calle", "avenida", "plaza", "parque", "jardin", "zoo", "museo", "teatro",
    "cine", "biblioteca", "escuela", "universidad", "profesor", "alumno", "examen", "tarea", "clase", "leccion",
    "conferencia", "seminario", "diploma", "certificado", "conocimiento", "aprendizaje", "investigacion",
    "descubrimiento", "invento", "tecnica", "tecnologia", "ingenieria", "avance", "innovacion", "progreso"
]


In [ ]:
#Fill DB
from concurrent.futures import ThreadPoolExecutor
from googletrans import Translator

# Variable global para el contador
global_counter = 0

def translate_and_add(word):
    global global_counter
    try:
        # Traducir la palabra al inglés
        translation = translator.translate(word, src='es', dest='en')
        translated_word = translation.text.lower()

        # Insertar la traducción en la tabla
        dictionary_db.addTranslation(WordData(word, "es-MX"), WordData(translated_word, "en-US"))

        # Incrementar el contador y mostrarlo
        global_counter += 1
        print(f"Palabra {global_counter}: '{word}' agregada y traducida a '{translated_word}'")
    except Exception as e:
        print(f"No se pudo insertar la palabra '{word}': {e}")

try:
    dictionary_db = DictionaryDB()
    cur = dictionary_db.getCursor()
    cur.execute("SELECT restartDictionary();")
    dictionary_db.connection.commit()
    dictionary_db.addLanguage("es-MX")
    dictionary_db.addLanguage("en-US")

    # Inicializa el traductor
    translator = Translator()

    palabras_unicas = list(set(words))
    print(f"Intentando agregar '{len(palabras_unicas)}' palabras")
    # Utiliza ThreadPoolExecutor para ejecutar las traducciones en paralelo
    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(translate_and_add, palabras_unicas)

except Exception as er:
    print(f"Error: {er}")
finally:
    del dictionary_db
